In [1]:
import pandas as pd
import torch
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from tensorflow.keras.metrics import Accuracy

2024-03-21 13:20:46.258907: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 13:20:46.301738: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 13:20:47.001966: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
classesDF  = pd.read_csv("/home/pfeliu/Desktop/Elliptic_Dataset/elliptic_txs_classes.csv")
edgesDF = pd.read_csv("/home/pfeliu/Desktop/Elliptic_Dataset/elliptic_txs_edgelist.csv")
featuresDF = pd.read_csv("/home/pfeliu/Desktop/Elliptic_Dataset/elliptic_txs_features.csv", header=None)
featuresDF.columns = ['txId', 'timestep'] + ['f' + str(i) for i in range(165)]

In [3]:
#class 2: LICIT // class 1: ILLICIT
classesDF['class'] = classesDF['class'].map({'2': 0, '1': 1, 'unknown': -1})

featuresDF = featuresDF.merge(classesDF, on='txId')

# Move features 'class' to first column
cols = list(featuresDF.columns)
cols = cols[:1] + [cols[-1]] + cols[1:-1]
featuresDF = featuresDF[cols]

featuresDF.head(5)

,txId,class,timestep,f0,f1,f2,f3,f4,f5,f6,...,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164
0,230425980,-1,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,-1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,-1,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,0,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,-1,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [4]:
len(featuresDF)

203769

In [5]:

X = featuresDF.drop(['txId', 'class'], axis=1).values.astype(float)
y = featuresDF['class'].values.astype(float)

is_labeled = y >= 0  # Esto seleccionará clases 1 y 2 como etiquetadas
is_unlabeled = y == -1  # Esto seleccionará clases -1 como no etiquetadas

X_labeled = X[is_labeled]
y_labeled = y[is_labeled]
print("y_labeled:", y_labeled)
X_unlabeled = X[is_unlabeled]

# Codificación One-hot de las etiquetas para los datos etiquetados
y_labeled_onehot = to_categorical(y_labeled - 1, num_classes=2)  # Ajustar las clases a 0 y 1

# División de los datos etiquetados en lícitos e ilícitos
X_licitos = X_labeled[y_labeled == 0]
print(len(X_licitos))
y_licitos = y_labeled_onehot[y_labeled == 0]

X_ilicitos = X_labeled[y_labeled == 1]
print(len(X_ilicitos))
y_ilicitos = y_labeled_onehot[y_labeled == 1]

#LICITOS
X_licitos_balanced = X_licitos[:4000]
print("X_licitos_balanced:",len(X_licitos_balanced))
X_licitos_eval = X_licitos[4000:]
print("X_licitos_eval: ", len(X_licitos_eval))

y_licitos_balanced= y_licitos[:4000]
print("y_licitos_balanced:",len(y_licitos_balanced))
y_licitos_eval = y_licitos[4000:]
print("y_licitos_balanced:",len(y_licitos_eval))

#ILICITOS
X_ilicitos_balanced = X_ilicitos[:4000]
print("X_ilicitos_balanced:",len(X_ilicitos_balanced))
X_ilicitos_eval = X_ilicitos[4000:]
print("X_ilicitos_eval: ", len(X_ilicitos_eval))

y_ilicitos_balanced= y_ilicitos[:4000]
print("y_ilicitos_balanced:",len(y_ilicitos_balanced))
y_ilicitos_eval = y_ilicitos[4000:]
print("y_ilicitos_balanced:",len(y_ilicitos_eval))

# Combinar y mezclar los datos balanceados para el entrenamiento
X_train_balanced = np.concatenate([X_licitos_balanced, X_ilicitos_balanced])
y_train_balanced = np.concatenate([y_licitos_balanced, y_ilicitos_balanced])
X_train_balanced, y_train_balanced = shuffle(X_train_balanced, y_train_balanced, random_state=42)

# Combinar los datos reservados para evaluación
X_eval = np.concatenate([X_licitos_eval, X_ilicitos_eval])
y_eval = np.concatenate([y_licitos_eval, y_ilicitos_eval])

# Asignar los datos no etiquetados como conjunto de test (para el autoentrenamiento o predicciones futuras)
X_test = X_unlabeled
print("x test / x unlabeled:",len(X_test)) #157205
X_test = X_test[:80000] #etiquetar solo 50.000 mas 
print("x test resize:",len(X_test)) #100000



y_labeled: [0. 0. 0. ... 1. 0. 1.]
42019
4545
X_licitos_balanced: 4000
X_licitos_eval:  38019
y_licitos_balanced: 4000
y_licitos_balanced: 38019
X_ilicitos_balanced: 4000
X_ilicitos_eval:  545
y_ilicitos_balanced: 4000
y_ilicitos_balanced: 545
x test / x unlabeled: 157205
x test resize: 80000


In [6]:
print(X_train_balanced.shape)
print(X_test.shape)
print(y_train_balanced.shape)



(8000, 166)
(80000, 166)
(8000, 2)


In [7]:
#print(X_train.shape)
#print(X_test.shape)
#print(y_train.shape)
#print(y_test.shape)
"""
(37251, 166)
(9313, 166)
(37251, 2)
(9313, 2)
"""


'\n(37251, 166)\n(9313, 166)\n(37251, 2)\n(9313, 2)\n'

In [8]:
# Definir el modelo
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_balanced.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 clases: lícitas e ilícitas

# Compilar el modelo
model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo con datos etiquetados
model.fit(X_train_balanced, y_train_balanced, epochs=100, batch_size=100)

# Predecir y evaluar con datos de prueba
#y_pred = model.predict(X_test)

"""
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
confusion_mtx = confusion_matrix(y_test_labels, y_pred_labels)
print(confusion_mtx)
"""


Epoch 1/100


2024-03-21 13:20:55.831071: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


80/80 [==============================] - 0s 2ms/step - loss: 0.1574 - accuracy: 0.9631
Epoch 2/100
80/80 [==============================] - 0s 1ms/step - loss: 0.0731 - accuracy: 0.9871
Epoch 3/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0610 - accuracy: 0.9876
Epoch 4/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0547 - accuracy: 0.9877
Epoch 5/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0505 - accuracy: 0.9880
Epoch 6/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0472 - accuracy: 0.9886
Epoch 7/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0446 - accuracy: 0.9886
Epoch 8/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0425 - accuracy: 0.9891
Epoch 9/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0406 - accuracy: 0.9893
Epoch 10/100
80/80 [==============================] - 0s 2ms/step - loss: 0.0390 - accuracy: 0.9893
Epoch 11/100
80/80 [=

'\ny_pred_labels = np.argmax(y_pred, axis=1)\ny_test_labels = np.argmax(y_test, axis=1)\nconfusion_mtx = confusion_matrix(y_test_labels, y_pred_labels)\nprint(confusion_mtx)\n'

In [9]:

# Función de auto-entrenamiento adaptada para datos tabulares
def self_training(model, X_labeled, y_labeled, X_unlabeled, threshold, epochs):
    for epoch in range(epochs):
        # Entrenar el modelo
        model.fit(X_labeled, y_labeled, epochs=5, batch_size=100)

        model.save(f"model_{epoch+1}.keras")

        # Predecir etiquetas para datos no etiquetados
        predictions = model.predict(X_unlabeled)
        confidences = np.max(predictions, axis=1)
        high_confidence_indices = confidences > threshold

        # Seleccionar datos con alta confianza
        X_high_confidence = X_unlabeled[high_confidence_indices]
        y_high_confidence = predictions[high_confidence_indices]

        # Actualizar conjuntos etiquetados
        X_labeled = np.concatenate([X_labeled, X_high_confidence])
        y_labeled = np.concatenate([y_labeled, y_high_confidence])

        # Eliminar los seleccionados de los datos no etiquetados
        X_unlabeled = np.delete(X_unlabeled, high_confidence_indices, axis=0)
        print(f"Iteración {epoch+1}, Datos añadidos: {len(X_high_confidence)}, Datos faltantes: {len(X_unlabeled)}")

    return model, X_labeled, y_labeled



In [10]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_labels = np.argmax(y_pred, axis=1)
    y_test_labels = np.argmax(y_test, axis=1)
    
    # Calcular la precisión y la matriz de confusión
    accuracy = accuracy_score(y_test_labels, y_pred_labels)
    confusion_mtx = confusion_matrix(y_test_labels, y_pred_labels)
    
    return accuracy, confusion_mtx

In [11]:
model_self_trained, X_self_labeled, y_self_labeled = self_training(model, X_train_balanced, y_train_balanced, X_test, threshold=0.9, epochs=100)

"""
# Evaluar el modelo después del autoentrenamiento
final_accuracy, final_confusion_mtx = evaluate_model(model_self_trained, X_test, y_test)
print("Después del autoentrenamiento:")
print("Precisión:", final_accuracy)
print("Matriz de confusión:\n", final_confusion_mtx)
"""

Epoch 1/5
80/80 [==============================] - 0s 2ms/step - loss: 0.0108 - accuracy: 0.9971
Epoch 2/5
80/80 [==============================] - 0s 1ms/step - loss: 0.0107 - accuracy: 0.9971
Epoch 3/5
80/80 [==============================] - 0s 1ms/step - loss: 0.0106 - accuracy: 0.9971
Epoch 4/5
80/80 [==============================] - 0s 2ms/step - loss: 0.0105 - accuracy: 0.9971
Epoch 5/5
2500/2500 [==============================] - 3s 1ms/step
Iteración 1, Datos añadidos: 70698, Datos faltantes: 9302
Epoch 1/5
787/787 [==============================] - 1s 2ms/step - loss: 0.0273 - accuracy: 0.9997
Epoch 2/5
787/787 [==============================] - 1s 2ms/step - loss: 0.0273 - accuracy: 0.9997
Epoch 3/5
787/787 [==============================] - 1s 2ms/step - loss: 0.0273 - accuracy: 0.9997
Epoch 4/5
787/787 [==============================] - 1s 2ms/step - loss: 0.0273 - accuracy: 0.9997
Epoch 5/5
291/291 [==============================] - 0s 995us/step
Iteración 2, Datos añadi

'\n# Evaluar el modelo después del autoentrenamiento\nfinal_accuracy, final_confusion_mtx = evaluate_model(model_self_trained, X_test, y_test)\nprint("Después del autoentrenamiento:")\nprint("Precisión:", final_accuracy)\nprint("Matriz de confusión:\n", final_confusion_mtx)\n'

In [12]:
print(X_self_labeled.shape) 
#deberian ser 146564 (el total de nodos menos los 57205 no etiquetados que no etiquetamos pq son muchos)
print(y_self_labeled.shape)



(78931, 166)
(78931, 2)


In [13]:
y_self_labeled

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       ...,
       [0.9004134 , 0.09958673],
       [0.09940247, 0.90059733],
       [0.9006368 , 0.09936319]], dtype=float32)

In [22]:
from keras.models import load_model

model_self_trained = load_model('model_10.keras')

In [23]:
acc, conf_mtrx = evaluate_model(model_self_trained, X_eval, y_eval)
#0.16852504926874806
print("accuracy:", acc)
print("confusion matrix:", conf_mtrx)

1206/1206 [==============================] - 1s 1ms/step
accuracy: 0.16567264806555337
confusion matrix: [[  545     0]
 [32175  5844]]
